# Lab 5

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-414/blob/master/book/labs/lab_05.ipynb)

## Submission requirements

1. Upload a screenshot of your map for each question.
2. Provide a link to your notebook on Colab. See instructions [here](https://geog-414.gishub.org/book/labs/instructions.html).

In [1]:
import geemap
import ee
ee.Initialize()

## Question 1

Visualize the [USGS Watershed Boundary Dataset](https://developers.google.com/earth-engine/datasets/catalog/USGS_WBD_2017_HUC04) with outline color only, no fill color.

In [4]:
m=geemap.Map(center=[40,-100],zoom=4)
collection= ee.FeatureCollection('USGS/WBD/2017/HUC04')
style= {'color': 'ff0000ff', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
m.addLayer(collection.style(**style), {}, 'HUC04')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [5]:
text = "Made by Zhanchao Yang"
m.add_text(text, fontsize=20, position='bottomright')

![](https://i.imgur.com/PLlNFq3.png)

## Question 2 

Filter the USGS Watershed Boundary dataset and select the watershed that intersects the county of your choice.

In [ ]:
# Add your code here.

![](https://i.imgur.com/F2QfqZu.png)

## Question 3

Clip the [USGS 3DEP 10m DEM](https://developers.google.com/earth-engine/datasets/catalog/USGS_3DEP_10m) with the watershed that intersects the county of your choice. Display the DEM with a proper color palette and color bar.

In [ ]:
# Add your code here.

![](https://i.imgur.com/okR39pf.png)

## Question 4

Use the [USGS National Land Cover Database](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2019_REL_NLCD) and [US Census States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States) to create a split-panel map for visualizing land cover change (2001-2019) for a US state of your choice. Make sure you add the NLCD legend to the map.

In [ ]:
# Add your code here.

![](https://i.imgur.com/Au7Q5Ln.png)

## Questions 5

Download OpenStreetMap data for a city of your choice and visualize the city boundary and restaurants in the city.

In [ ]:
# Add your code here.

![](https://i.imgur.com/AUlO1CV.png)